In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' #默认为’last’

import pandas as pd
from gmplot import gmplot
import numpy as np
import random
random.seed(10)
from geopy import distance
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
import matplotlib.pyplot as plt
import glob
import os
import geopy.distance
from dipy.segment.metric import Metric
from dipy.segment.metric import mdf
from dipy.segment.metric import ResampleFeature

from dipy.segment.clustering import QuickBundles
from dipy.segment.clustering import QuickBundlesX
from gmplot import gmplot
from calculateTime import CalculateTime
import math
from fastdtw import fastdtw
from fastdtw import dtw

In [2]:
def calT(t1,t2):  #计算时间差
    return (CalculateTime(t2)-CalculateTime(t1)).seconds


def calDis(first,second):   ##计算轨迹点间距离
    return distance.distance(first,second).meters

def seg(dataFra,T=30):   #根据阈值长度，两点间距离截取单个轨迹段
    pointset=[]
    for i in range(dataFra.shape[0]-1):
        first=(dataFra.loc[i,['LAT']].LAT,dataFra.loc[i,['LON']].LON)
        second=(dataFra.loc[i+1,['LAT']].LAT,dataFra.loc[i+1,['LON']].LON)
        dis=calDis(first,second)
        if dis<40:
            pointset.append(i)
    
    segTraSet=[]
    if len(pointset)==0:
        segTraSet.append(dataFra)
        return segTraSet
    
    init=0
    for i in pointset:
        tmp=dataFra.iloc[init:i+1,:]
        if tmp.shape[0]>T:
            segTraSet.append(tmp)
        init=i+1
    tmp=dataFra.iloc[init:,:]
    if tmp.shape[0]>T:
            segTraSet.append(tmp)
    return segTraSet

In [3]:
def segMMSI(path):         ##根据MMSI号分离轨迹
    AisData=pd.read_csv(path)
    mmsi=AisData['MMSI'].unique()
    fdata=[]
    for i,mi in enumerate(mmsi):
        temp=AisData[AisData['MMSI']==mmsi[i]]
        temp=temp.sort_values(by=['BaseDateTime'])
        temp.index=list(range(len(temp)))

        fdata.append(temp)
    return fdata

In [4]:
def segAll(fdata,T=30):        ##按距离分离全部轨迹段
    segFinal=[]
    for data in fdata:
        sub=seg(data)
        for item in sub:
            if len(item)>T:
                item.index=range(len(item))
                segFinal.append(item)
    return segFinal

In [5]:
#按时间间隔与位置点容量  分离轨迹
def segTime(dataFra,T=30):
    pointset=[]
    for i in range(dataFra.shape[0]-1):
        t1=CalculateTime(dataFra.loc[i,['BaseDateTime']].BaseDateTime)
        t2=CalculateTime(dataFra.loc[i+1,['BaseDateTime']].BaseDateTime)
        delta_time=(t2-t1).seconds
        
        if delta_time>600:
            pointset.append(i)
    segTraSet=[]
    if len(pointset)==0:
        segTraSet.append(dataFra)
        return segTraSet
    init=0
    for i in pointset:
        tmp=dataFra.iloc[init:i+1,:]
        if tmp.shape[0]>T:
            segTraSet.append(tmp)
        init=i+1
    tmp=dataFra.iloc[init:,:]
    if tmp.shape[0]>T:
            segTraSet.append(tmp)
    return segTraSet

In [6]:
def segTimeALL(fdata,T=30):
    segFinal=[]
    for data in fdata:
        sub=segTime(data)
        for item in sub:
            if len(item)>T:
                item.index=range(len(item))
                segFinal.append(item)
    return segFinal

In [2]:
def randomcolor():
    colorArr = ['1','2','3','4','5','6','7','8','9','A','B','C','D','E','F']
    color = ""
    for i in range(6):
        color += colorArr[random.randint(0,14)]
    return "#"+color

In [8]:
def dtw_distance(ts_a, ts_b, d=lambda x,y: geopy.distance.distance([x[0], x[1]], [y[0], y[1]]).kilometers, mww=10000):
    """Computes dtw distance between two time series
    
    Args:
        ts_a: time series a
        ts_b: time series b
        d: distance function
        mww: max warping window, int, optional (default = infinity)
        
    Returns:
        dtw distance
    """
    
    # Create cost matrix via broadcasting with large int
    ts_a, ts_b = np.array(ts_a), np.array(ts_b)
    M, N = len(ts_a), len(ts_b)
    cost = np.ones((M, N))

    # Initialize the first row and column
    cost[0, 0] = d(ts_a[0], ts_b[0])
    for i in range(1, M):
        cost[i, 0] = cost[i-1, 0] + d(ts_a[i], ts_b[0])

    for j in range(1, N):
        cost[0, j] = cost[0, j-1] + d(ts_a[0], ts_b[j])

    # Populate rest of cost matrix within window
    for i in range(1, M):
        for j in range(max(1, i - mww), min(N, i + mww)):
            choices = cost[i-1, j-1], cost[i, j-1], cost[i-1, j]
            cost[i, j] = min(choices) + d(ts_a[i], ts_b[j])

    # Return DTW distance given window 
    return cost[-1, -1]

In [9]:
path="C:\\Code\\毕业设计\\轨迹聚类\\data\\WGH\\*csv"
pathls=glob.glob(path)

In [10]:
fdata=segMMSI(pathls[0])
segFinal=segAll(fdata)
segTimeFinal=segTimeALL(segFinal)

In [11]:
for p in pathls[1:]:
    segTimeFinal.extend(segTimeALL(segAll(segMMSI(p))))
for i in range(len(segTimeFinal)):
    segTimeFinal[i]['BaseDateTime']=segTimeFinal[i]['BaseDateTime'].apply(CalculateTime)

C:\software\Anaconda3\envs\BY\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
streams=[]
streams_time=[]
for i in range(len(segTimeFinal)):
   
    tmp_time=np.array(segTimeFinal[i].loc[:,['LAT','LON','MMSI']])
    tmp=np.array(segTimeFinal[i].loc[:,['LAT','LON']])
    streams.append(tmp)
    streams_time.append(tmp_time)
streams=np.array(streams)
streams_time=np.array(streams_time)

**分类+展示**

In [124]:
%%time
THRESHOLD =5 #越高分类数越少
class GPSDistance(Metric):
    def __init__(self):
        super(GPSDistance, self).__init__(feature=ResampleFeature(nb_points=50))

    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)

#     def dist(self, v1, v2):
#         x = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2))]
#         currD = np.mean(x)
#         return currD
    
#     def dist(self, v1, v2):
#          #3
#         currD=dtw(v1,v2)[0]
#         return currD
    
#     def dist(self,v1,v2):
#         #MDF
#         x1 = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2))]
#         d_direct=np.mean(x1)
#         x2= [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2[::-1]))]
#         d_flipped=np.mean(x2)
#         currD=min(d_direct,d_flipped)
#         return currD

#     def dist(self,v1,v2):
#         #MDF 4
#         x1 = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2))]
#         d_direct=np.mean(x1)
#         x2= [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2[::-1]))]
#         d_flipped=np.mean(x2)
#         currD=min(d_direct,d_flipped)
#         return currD

    def dist(self, v1, v2):
        #5.5
        currD=mdf(v1,v2)
        return currD*100
    
metric = GPSDistance()
qb = QuickBundles(threshold=THRESHOLD,metric=metric)

clusters = qb.cluster(streams)
print("Nb. clusters:", len(clusters))

Nb. clusters: 17
Wall time: 265 ms


In [125]:
gmap = gmplot.GoogleMapPlotter(streams[0][0,0], streams[0][0,1], 12)
# MM=[]

for clustersIndex in range(len(clusters)):
    color = randomcolor()
#     if len(clusters[clustersIndex].indices)==18:
#            continue
#     if len(clusters[clustersIndex].indices)==20:
#            continue        
#     if len(clusters[clustersIndex].indices) in changdu:
#             continue
#     alltra=[]
    for i in clusters[clustersIndex].indices:
#         MM.append(i)
        gmap.plot(streams[i][:,0], streams[i][:,1], color, edge_width=1)
#         tocsv.append(streams_time[i])
#         alltra.append(streams[i])
#     np.save(f"../HtmlMap/9month/{len(alltra)}.npy",np.array(alltra))
gmap.draw("../HtmlMap/WGH9_Clu_all.html")
# gmap.draw("../HtmlMap/WGH_Clu_3-6_MDF_f5.5.html")

In [96]:
gmap = gmplot.GoogleMapPlotter(streams[0][0,0], streams[0][0,1], 12)

for clustersIndex in range(len(clusters)):
    color = randomcolor()
#     if len(clusters[clustersIndex].indices)<30:
#             continue
#     if len(clusters[clustersIndex].indices) in changdu:
#             continue
    for i in clusters[clustersIndex].indices:
        gmap.plot(streams[i][:,0], streams[i][:,1], color, edge_width=1)
#         tocsv.append(streams_time[i])
    gmap.draw(f"../HtmlMap/910-930/WGH-{len(clusters[clustersIndex].indices)}.html")

**展示**

**单条**显示

In [116]:
gmap = gmplot.GoogleMapPlotter(streams[0][0,0], streams[0][0,1], 12)
for i in range(len(clusters[8])):
    if 5<i <10:
        gmap.plot(clusters[8][i][:,0],clusters[8][i][:,1], randomcolor(), edge_width=1)

gmap.draw("../HtmlMap/WGH_sub1.html")